# Régime d'Assurance Chômage (RAC)

## Simulation RAC sans retraite anticipée

In [1]:
# Chargement des librairies
from cnaclib.rac import RAC
import pandas as pd
from datetime import datetime
import os

In [2]:
help(RAC)

Help on class RAC in module cnaclib.rac:

class RAC(builtins.object)
 |  RAC(DateRecrutement, DateCompression, SMM)
 |  
 |  REGIME ASSURANCE CHOMAGE : SIMULATEUR
 |  
 |  Cette Class en python permet de réaliser des simulations pour le calculs des différents éléments liés au régime d'assurance chômage.
 |  Elle permet de :
 |  - Vérifier la condition d'admission relative à l'experience professionnelle;
 |  - Calculer la durée de prise en charge (DPC);
 |  - Calculer le montant de la Contribution d'Ouverture de Droits;
 |  - Récupérer le montant du SNMG en fonction de la date;
 |  - Calculer les montants d'indemnités en fonction des 04 périodes;
 |  - Calculer les montants de cotisations de sécurité sociale (part patronale & part salariale );
 |  
 |  Parameters
 |  ----------
 |  
 |  DateRecrutement : date, 
 |      C'est de la date de recrutement du salarié chez le dernier employeur.
 |      Elle doit être exprimé selon le format : dd/mm/yyyy.
 |  
 |  
 |  DateCompression : date,
 

In [39]:
# Déclaration des paramètres

dateRecrutement = '11/02/2021'
dateCompression =  '15/02/2021'
smm = float(60000)


In [40]:
# Instanciation de l'objet simulation
simul = RAC(dateRecrutement, dateCompression, smm) 

In [41]:
# Calcul de la durée d'experience 
dureeExperienceAnnee = simul.durexp()[0]
dureeExperienceMois = simul.durexp()[1]
dureeExperienceJours = simul.durexp()[2]
dureeExperienceValidee = f"Votre expérience calculée est égale à {dureeExperienceAnnee} an(s), {dureeExperienceMois} moi(s) & {dureeExperienceJours} jour(s)"

print(dureeExperienceValidee)

Votre expérience calculée est égale à 0 an(s), 0 moi(s) & 4 jour(s)


In [42]:
# Vérification de l'admission
admission = simul.admission()[0]

print(admission)

Selon votre expérience calculée, vous ne pouvez pas bénéficier du RAC 


In [43]:
# Calcul de la date d'admission (théorique)
dateAdmission = simul.admission()[1].strftime("%d/%m/%Y")

print(dateAdmission)

15/03/2021


In [44]:
# Calcul de la durée de prise en charge en mois
dpc = simul.DPC()

print(str(dpc))

None


In [49]:
# Calcul de la COD mensuel
codMensuel = simul.COD()[0]

print(codMensuel)

Selon votre expérience calculée, vous ne pouvez pas bénéficier du RAC


NameError: name 'exit' is not defined

In [50]:
# Calcul de la COD totale
codTotale = simul.COD()[1]

print(codTotale)

Selon votre expérience calculée, vous ne pouvez pas bénéficier du RAC


NameError: name 'exit' is not defined

In [51]:
# Calcul de la date de fin de droit
dateFDD = simul.Date_FDD().strftime("%d/%m/%Y")

print(dateFDD)

TypeError: unsupported operand type(s) for +: 'method' and 'int'

In [52]:
# Afficher le calendrier des indemnités 
tableau = simul.tableaux_Indemnites()
tableau

TypeError: unsupported operand type(s) for +: 'method' and 'int'